In [1]:
import pandas as pd
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from tqdm.auto import tqdm

In [2]:
# Подключаемся к локальному Qdrant
client = QdrantClient("localhost", port=6333)

# КОНФИГ
COLLECTION_NAME = "nlp2025_chunks"
VECTOR_SIZE = 1024
BATCH_SIZE = 500

In [ ]:
# 1. Проверяем и удаляем, если уже есть (чтобы начать с чистого листа)
if client.collection_exists(collection_name=COLLECTION_NAME):
    client.delete_collection(collection_name=COLLECTION_NAME)
    print(f"Коллекция '{COLLECTION_NAME}' удалена.")

# 2. Создаем заново
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE, 
        distance=Distance.COSINE
    )
)
print(f"Коллекция '{COLLECTION_NAME}' успешно создана! Готовы к заливке.")

In [ ]:
# Путь к файлу
parquet_file = "../data/processed/all_chunks_with_embeddings.parquet"

print("Читаем Parquet файл...")
df = pd.read_parquet(parquet_file)
print(f"Загружено строк: {len(df)}")

points_batch = []
total_records = len(df)

# Итерируемся по датафрейму
print("Начинаем заливку в Qdrant...")
for i, row in tqdm(df.iterrows(), total=total_records, desc="Uploading"):
    
    # Конвертация numpy array -> list float32
    # Это критично, чтобы не словить ошибку сериализации JSON
    vector = row["embedding"].astype(np.float32).tolist()
    
    # Сборка метаданных
    payload = {
        "text": row["text"],
        "source": row["source_path"],
        # Распаковка остальных метаданных, если есть
        **(row["metadata"] if row["metadata"] else {}) 
    }

    # Добавляем точку в буфер
    points_batch.append(
        PointStruct(
            id=i,  # Используем индекс DataFrame как ID
            vector=vector,
            payload=payload
        )
    )

    # Если буфер заполнился — отправляем
    if len(points_batch) >= BATCH_SIZE:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=points_batch,
            wait=False # Не ждем подтверждения записи на диск (быстрее)
        )
        points_batch = [] # Очищаем буфер

# Дозаливаем остатки, если буфер не пуст
if points_batch:
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=points_batch,
        wait=True # Тут ждем, чтобы убедиться, что всё закончилось
    )

print("Заливка завершена!")

In [ ]:
# Тестовый поиск (берем вектор из датасета для примера)
test_vector = df.iloc[0]["embedding"].astype(np.float32).tolist()

hits = client.query_points(
    collection_name=COLLECTION_NAME,
    query=test_vector,
    limit=3
)

print("\nТестовый поиск (Top-3):")
for hit in hits.points:
    print(f"ID: {hit.id}, Score: {hit.score}")
    if hit.payload:
        print(f"Payload: {hit.payload}")
    print("---")

In [5]:
synthetic_vector = np.random.uniform(low=-1.0, high=1.0, size=(VECTOR_SIZE,)).tolist()

In [8]:
search_result = client.query_points(
    collection_name=COLLECTION_NAME,
    query=synthetic_vector,
    limit=5,
    with_payload=True,
    with_vectors=False
)

In [17]:
search_result

QueryResponse(points=[ScoredPoint(id=311079, version=623, score=0.12865478, payload={'text': 'RSCF: Relation-Semantics Consistent Filter for Entity Embedding of Knowledge Graph: ###### Abstract  \nIn knowledge graph embedding, leveraging relation specific entity transformation has markedly enhanced performance. However, the consistency of embedding differences before and after transformation remains unaddressed, risking the loss of valuable inductive bias inherent in the embeddings.\nThis inconsistency stems from two problems. First, transformation representations are specified for relations in a disconnected manner, allowing dissimilar transformations and corresponding entity embeddings for similar relations.\nSecond, a generalized plug-in approach as a SFBR (Semantic Filter Based on Relations) disrupts this consistency through excessive concentration of entity embeddings under entity-based regularization, generating indistinguishable score distributions among relations.\nIn this pape